In [18]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
import os
import re
import pandas as pd
from bs4 import BeautifulSoup
import azapi

In [3]:
## CLIENT ID AND CLIENT SECRET
### DO NOT PUT ON GIT

client_id = '56b2fcfa2d4e49dbb98250c7ad978501'
client_secret = '5eb60a3ebfc741ecaf96112d9e55dc7a'
genius_access_token = 'SgMlTfZqSJurKYld58zSWR_k8tFu2Eckd-WEuPuKIdanWHP5k3OyzILuMCeLyY6P'


In [4]:
def get_top_tracks(artist_name, client_id, client_secret):
    # Initialize Spotify client with your credentials
    credentials = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
    sp = spotipy.Spotify(client_credentials_manager=credentials)
    
    # search for the artist to get the Spotify ID through sp.search
    results = sp.search(q='artist:' + artist_name, type='artist') 
    items = results['artists']['items']
    if not items:
        return []
    
    
    artist_id = items[0]['id']
    
    top_tracks = sp.artist_top_tracks(artist_id)
    
    return [track['name'] for track in top_tracks['tracks']]

In [5]:
artist_name = 'Selena Gomez'
top_tracks = get_top_tracks(artist_name, client_id, client_secret)
top_tracks

['Calm Down (with Selena Gomez)',
 'Lose You To Love Me',
 'Single Soon',
 'It Ain’t Me (with Selena Gomez)',
 'Love On',
 'People You Know',
 'Wolves',
 'Back To You - From 13 Reasons Why – Season 2 Soundtrack',
 'Fetish (feat. Gucci Mane)',
 'Same Old Love']

In [29]:
def get_lyrics(song_name, artist_name, genius_access_token):
    #set up the request using access token 
    base_url = "https://api.genius.com"
    headers = {'Authorization': 'Bearer ' + genius_access_token}
    search_url = base_url + "/search" #getting ready to search!
    data = {'q': song_name} #we want to search the song name!

    response = requests.get(search_url, params=data, headers=headers)
    json = response.json()
    path = None

    # loops through the search results returned by the Genius API to find a specific song that matches name of song and artist name

    for hit in json["response"]["hits"]:
        if artist_name.lower() in hit["result"]["primary_artist"]["name"].lower():
            path = hit["result"]["path"]
            break #assuming the first is right 

    if path:
        lyrics_url = "https://genius.com" + path
        lyrics_response = requests.get(lyrics_url)
        soup = BeautifulSoup(lyrics_response.text, 'lxml')

        # Check if the song is instrumental
        instrumental_div = soup.find('div', class_='LyricsPlaceholder__Message-uen8er-2')
        if instrumental_div:
            return "This song is an instrumental"

        lyrics_div = soup.find('div', {'data-lyrics-container': 'true'})
        lyrics = lyrics_div.get_text(separator='\n', strip=True) if lyrics_div else "Lyrics not found"
        return lyrics

    else:
        return "Lyrics not found"

In [24]:
API = azapi.AZlyrics(accuracy=0.5)
def get_lyrics(song, artist):
    API.artist = artist
    API.title = song
    lyrics = API.getLyrics(save=False, ext='lrc')
    #print(API.title, API.artist)
    if lyrics == 0:
        return "Lyrics not found"
    else:
        return lyrics

In [28]:
lyrics = get_lyrics('intro (end of the world)', 'Ariana Grande')
lyrics

Error no metadata


1

In [15]:
cwd = os.getcwd()
cwd
viral_songs = f'{cwd}/viral.csv'
viral_songs = pd.read_csv(viral_songs)
viral_songs

,artists,songs
0,Kinfolk Thugs,Dumptruck
1,Cordelia,Little Life
2,"¥$, Kanye West, Ty Dolla $ign & Rich The Kid",CARNIVAL (feat. Playboi Carti)
3,Ariana Grande,"yes, and? (Mixed)"
4,Bobby Caldwell,What You Won't Do for Love
5,R.m.y,How I Love Being a Woman 6
6,Rihanna,Bitch Better Have My Money
7,"The Weeknd, JENNIE & Lily Rose Depp",One Of The Girls
8,Gold-Tiger,Funny
9,Beyoncé,Countdown


In [16]:
from collections import Counter

singers = []
artists = Counter(viral_songs['artists'])
for artist in artists: 
    singers.append(artist)

singers

['Kinfolk Thugs',
 'Cordelia',
 '¥$, Kanye West, Ty Dolla $ign & Rich The Kid',
 'Ariana Grande',
 'Bobby Caldwell',
 'R.m.y',
 'Rihanna',
 'The Weeknd, JENNIE & Lily Rose Depp',
 'Gold-Tiger',
 'Beyoncé',
 'Masego & FKJ',
 'Lana Del Rey',
 'Peso Pluma & Anitta',
 '21 Savage',
 'Flo Milli',
 'Muni Long',
 'ผดุง ทรงแสง (แจ๊ส)',
 'Lilithzplug']

In [19]:
for singer in singers: 
    full_artists = [x.strip() for artist in artists for x in re.split(r',|&', artist)]
full_artists

['Kinfolk Thugs',
 'Cordelia',
 '¥$',
 'Kanye West',
 'Ty Dolla $ign',
 'Rich The Kid',
 'Ariana Grande',
 'Bobby Caldwell',
 'R.m.y',
 'Rihanna',
 'The Weeknd',
 'JENNIE',
 'Lily Rose Depp',
 'Gold-Tiger',
 'Beyoncé',
 'Masego',
 'FKJ',
 'Lana Del Rey',
 'Peso Pluma',
 'Anitta',
 '21 Savage',
 'Flo Milli',
 'Muni Long',
 'ผดุง ทรงแสง (แจ๊ส)',
 'Lilithzplug']

In [22]:
for artist in full_artists: 
    tracks = get_top_tracks(artist, client_id, client_secret)
    print(artist)
    for track in tracks:
        print(track)
        lyrics = get_lyrics(track, artist)

Kinfolk Thugs
Back It up and Dump It (Dump Truck)
Google found nothing!
Dumptruck
Google found nothing!
Dump Truck
Google found nothing!
Dump Truck Remix (Clean)
Google found nothing!
I Speak Memphis
Google found nothing!
Who Gon Check It
Similarity < 0.5
Dump Truck Remix (Explicit)
Google found nothing!
Back Seat (Pull It Over)
Google found nothing!
Money Maker
Similarity < 0.5
Dump Truck (Back it up and Dump it) - instrumental
Google found nothing!
Cordelia
Little Life
I Just Killed a Spider
Google found nothing!
Caramel
Google found nothing!
Play Pretend
What I Wouldn't Do
Google found nothing!
Little Life - radio edit cos they made me
Google found nothing!
Not So Sure
¥$
CARNIVAL
Similarity < 0.5
BURN
Similarity < 0.5
FUK SUMN
Similarity < 0.5
BACK TO ME
Similarity < 0.5
DO IT
Similarity < 0.5
STARS
Similarity < 0.5
TALKING
Similarity < 0.5
PAID
Similarity < 0.5
PAPERWORK
Similarity < 0.5
VULTURES
Similarity < 0.5
Kanye West
CARNIVAL
Similarity < 0.5
Heartless
Flashing Lights
Fathe

KeyboardInterrupt: 

In [ ]:
import json

lyrics = {}
for artist in full_artists:
    lyrics[artist] = {'songs': []}
    tracks = get_top_tracks(artist, client_id, client_secret)
    for track in tracks:
        song_lyrics = get_lyrics(track, artist, genius_access_token)
        lyrics[artist]['songs'].append({
            'song_name': track,
            'lyrics': song_lyrics
        })

output_file = "top_lyrics.json"

with open(output_file, "w") as json_file:
    json.dump(lyrics, json_file)

print("Lyrics saved to", output_file)